In [1]:
#data manipulation
import pandas as pd
import numpy as np
from tqdm import tqdm

#path configuration
import os
import sys

#import config to obtain paths
sys.path.insert(-2, '../..')
from config import *

#import input path
path_input_confusion_matrix = path_confusion_matrices

#import output path
path_output = path_performance_indicators

#functions to the app
from functions_bootstrapping_part import *

np.random.seed(121) #generate random seed

In [6]:
distribution = "_cmpoisson"
scenario = "sce1"
distribution_name = "CMP"
index_boots = 3
df_spi, df_tpi, df_dpi, df_min, df_max, df_pdt, df_avg, df_med = import_confusion_matrix(path_input_confusion_matrix, scenario, distribution)

In [22]:
index_boots = 2
df_spi_perm = df_spi.sample(frac = 1.0, axis = 1, random_state = index_boots, replace = True)
df_spi_perm
len(df_spi_perm.columns.value_counts().index)

6381

In [16]:
len(df_spi_perm.columns.value_counts().index)

6354

In [23]:
df_spi_perm

,sim_7337,sim_2576,sim_6638,sim_2515,sim_1100,sim_4771,sim_434,sim_6752,sim_2774,sim_5168,...,sim_2382,sim_7552,sim_3075,sim_8169,sim_994,sim_6269,sim_1120,sim_3656,sim_8486,sim_2590
1,TP,TN,TP,FN,TN,FN,FN,FN,TN,TN,...,FN,FN,FN,FN,FN,FN,TN,FN,FN,TN
2,TP,TN,TP,FN,TN,FN,FN,FN,TN,TN,...,FN,FN,FN,FN,FN,FN,TN,FN,FN,TN
3,TP,TN,TP,FN,TN,FN,FN,FN,TN,TN,...,FN,FN,FN,FN,FN,FN,TN,FN,FN,TN
4,TP,TN,TP,FN,TN,FN,FN,FN,TN,TN,...,FN,FN,FN,FN,FN,FN,TN,FN,FN,TN
5,TP,TN,TP,FN,TN,FN,FN,FN,TN,TN,...,FN,FN,FN,FN,FN,FN,TN,FN,FN,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,TP,TN,FN,FN,TN,FN,FN,TP,TN,TN,...,TP,TP,FN,FN,TP,TP,TN,FN,FN,TN
327,TP,TN,FN,FN,TN,FN,FN,TP,TN,TN,...,TP,TP,FN,FN,TP,TP,TN,FN,FN,TN
328,TP,TN,FN,FN,TN,FN,FN,TP,TN,TN,...,TP,TP,FN,FN,FN,TP,TN,FN,FN,TN
329,TP,TN,FN,FN,TN,FN,FN,TP,TN,TN,...,TP,TP,FN,FN,FN,TP,TN,FN,FN,TN


In [ ]:
for distribution in ["_cmpoisson"]: 

    if distribution == "":
        distribution_name = "Triangular"
    if distribution == "_gamma":
        distribution_name = "Gamma"
    if distribution == "_cmpoisson":
        distribution_name = "CMP"

    for scenario in ["sce1"]:

        print (f"Executando simulação para a distribuição {distribution_name} e cenário {scenario}... ")

        #Import dos dataframes do cenário
        df_spi, df_tpi, df_dpi, df_min, df_max, df_pdt, df_avg, df_med = import_confusion_matrix(path_input_confusion_matrix, scenario, distribution)

        #Criação das listas auxiliares
        list_df = [df_spi, df_tpi, df_dpi, df_min, df_max, df_pdt, df_avg, df_med]
        list_name = ["spi", "tpi", "dpi", "min_t", "max_t", "pdt_t", "avg_t", "med_t"]

        for df, indicator_name in zip(list_df, list_name):

            #slice the df from 20 to 20 review periods
            df_sliced = slice_df_20(df)

            #criação do resampling:
            for index_boots in tqdm(range (300), desc = f"distribution: {distribution_name} | scenario: {scenario} | indicator {indicator_name}"):

                #resampling
                df_sampled = df_sliced.sample(frac = 0.8, axis = 1, random_state = index_boots)

                #create temporary dataframes with performance metrics
                tmp_project_level = calculate_project_level_metrics(df_sampled)
                tmp_period_level = calculate_period_level_metrics(df_sampled)

                #concatenate and adjust dataframes
                tmp = concatenate_adjust_df(tmp_project_level, tmp_period_level, scenario, indicator_name, distribution_name, index_boots)

                #concatenate all 
                if (indicator_name == "spi") and (scenario == "sce1") and (distribution_name == distribution_name) and (index_boots == 0):
                    df_final = tmp.copy()
                else:
                    df_final = pd.concat([df_final, tmp])

        print("\n")    

print ("Simulação concluida")